## Prepare the Data

In [1]:
# Set start time

from datetime import datetime
start_time = datetime.now()

In [1]:
# Import the data
from tensorflow.keras import layers, models, Model
import pandas as pd
import numpy as np
import re
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import balanced_accuracy_score
# from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, LabelEncoder
# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import StandardScaler


In [2]:
  # extract relevant columns

relevant_df = df[['ID','County', 'Severity', 'Start_Time','End_Time',
                  'Weather_Condition']]

relevant_df['Start_Time'] = pd.to_datetime(relevant_df['Start_Time'])
relevant_df['End_Time'] = pd.to_datetime(relevant_df['End_Time'])
# relevant_df['Year'] = relevant_df['Start_Time'].dt.year
relevant_df['Start_Time_int'] = relevant_df['Start_Time'].dt.strftime('%Y%m%d').astype(int)
relevant_df['crash_duration'] = (relevant_df['End_Time'] - relevant_df['Start_Time'])
relevant_df['crash_duration'] = relevant_df['crash_duration']/np.timedelta64(1,'ns')
# relevant_df['Crash'] = relevant_df.apply(lambda x : 1 if x['Severity'] == '4' else 0, axis=1)


#relevant_df = df[['ID', 'Start_Time', 'City', 'State', 'County', 'Airport_Code', 'End_Time', 'Timezone', 'Zipcode', 'Description', 'Severity',
#'Temperature(F)', 'Distance(mi)', 'Wind_Speed(mph)', 'Precipitation(in)',
#'Weather_Condition', 'Weather_Timestamp']]

[print('accident_count = ', relevant_df.count())]


                    Country Name Country Code      2001       2002       2003  \
0                          Aruba          ABW  4.182003  -0.944955   1.110505   
1    Africa Eastern and Southern          AFE  3.489314   3.818941   2.934246   
2                    Afghanistan          AFG       NaN        NaN   8.832278   
3     Africa Western and Central          AFW  5.203518   9.918281   5.535575   
4                         Angola          AGO  4.205999  13.665689   2.990000   
..                           ...          ...       ...        ...        ...   
261                       Kosovo          XKX       NaN        NaN        NaN   
262                  Yemen, Rep.          YEM  3.803646   3.935232   3.747398   
263                 South Africa          ZAF  2.700000   3.700374   2.949075   
264                       Zambia          ZMB  5.316868   4.506014   6.944974   
265                     Zimbabwe          ZWE  1.439615  -8.894024 -16.995075   

          2004       2005  

In [ ]:
# Create DataFrame that containes only Colorado

CO_crashes_df =  relevant_df.loc[(df["State"] == "CO")]
CO_crashes_df['Severity'].value_counts()

In [3]:
# Update CO_crashes_df with numeric county code

county_dict = {
"Adams":1,"Alamosa":3,"Arapahoe":5,"Archuleta":7,"Baca":9,"Bent":11,"Boulder":13,"Broomfield":14,"Chaffee":15,"Cheyenne":17,"Clear Creek":19,"Conejos":21,"Costilla":23,"Crowley":25,"Custer":27,
"Delta":29,"Denver":31,"Dolores":33,"Douglas":35,"Eagle":37,"El Paso":41,"Elbert":39,"Fremont":43,"Garfield":45,"Gilpin":47,"Grand":49,"Gunnison":51,"Hinsdale":53,"Huerfano":55,
"Jackson":57,"Jefferson":59,"Kiowa":61,"Kit Carson":63,"La Plata":67,"Lake":65,"Larimer":69,"Las Animas":71,"Lincoln":73,"Logan":75,"Mesa":77,"Mineral":79,
"Moffat":81,"Montezuma":83,"Montrose":85,"Morgan":87,"Otero":89,"Ouray":91,"Park":93,"Phillips":95,"Pitkin":97,"Prowers":99,"Pueblo":101,"Rio Blanco":103,"Rio Grande":105,"Routt":107,
"Saguache":109,"San Juan":111,"San Miguel":113,"Sedgwick":115,"Summit":117,"Teller":119,"Washington":121,"Weld":123,"Yuma":125
}

CO_crashes_df_updated =  CO_crashes_df.replace({"County": county_dict})

CO_crashes_df_updated.head(5)


,Country Name,Country Code,2001,2002,2003,2004,2005,2006,2007,2008,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,Aruba,ABW,4.182003,-0.944955,1.110505,7.293729,-0.383140,1.127412,3.089544,1.835756,...,-1.586575,-0.623626,1.719624,7.048533,2.381730,-2.302837,-23.982580,27.639357,10.458317,NaN
1,Africa Eastern and Southern,AFE,3.489314,3.818941,2.934246,5.547088,6.158693,6.578271,6.646334,4.355519,...,3.983296,3.004495,2.172836,2.641704,2.681423,2.187553,-2.771340,4.504300,3.571318,NaN
2,Afghanistan,AFG,NaN,NaN,8.832278,1.414118,11.229715,5.357403,13.826320,3.924984,...,2.724543,1.451315,2.260314,2.647003,1.189228,3.911603,-2.351101,-20.738839,NaN,NaN
3,Africa Western and Central,AFW,5.203518,9.918281,5.535575,7.996554,5.824941,5.374564,5.524314,6.279592,...,5.926266,2.813543,0.142092,2.286270,2.838829,3.200919,-0.938162,3.976317,3.766354,NaN
4,Angola,AGO,4.205999,13.665689,2.990000,10.950000,15.030000,11.550000,14.010000,11.170000,...,4.822559,0.943572,-2.580111,-0.147150,-1.316362,-0.702273,-5.638215,1.199211,3.045403,NaN
5,Albania,ALB,8.293313,4.536524,5.528637,5.514668,5.526424,5.902659,5.983260,7.500041,...,1.774449,2.218726,3.314981,3.802599,4.019346,2.087712,-3.302082,8.908528,4.856402,NaN
6,Andorra,AND,8.119358,4.546362,8.694204,8.135676,5.397796,4.808689,1.553188,-5.559186,...,2.504466,1.434140,3.709678,0.346072,1.588765,2.015548,-11.183940,8.287200,8.809579,NaN
7,Arab World,ARB,1.660912,0.627321,4.889134,8.905290,5.499289,6.114007,4.528793,5.568095,...,2.404063,3.464341,3.688971,1.246423,2.505216,1.403734,-5.065286,3.800793,6.114674,NaN
8,United Arab Emirates,ARE,1.399085,2.433457,8.800541,9.566437,4.855141,9.837320,3.184390,3.191836,...,4.165692,6.786773,5.561491,0.735069,1.313914,1.108348,-4.957052,4.354755,7.850418,NaN
9,Argentina,ARG,-4.408840,-10.894485,8.837041,9.029573,8.851660,8.047152,9.007651,4.057233,...,-2.512615,2.731160,-2.080328,2.818503,-2.617396,-2.000861,-9.900485,10.718010,4.956370,NaN


In [ ]:
CO_crashes_df_updated['Weather_Condition'] = CO_crashes_df_updated['Weather_Condition'].apply(lambda x: re.sub(r'[^a-zA-Z\s]','_',str(x)))

CO_crashes_df_updated['Weather_Condition'] = CO_crashes_df_updated['Weather_Condition'].replace({'Overcast': 'Cloudy', 
                                                                'Light Freezing Drizzle': 'Light Drizzle', 
                                                                'Snow _ Windy':'Light Snow _ Windy',
                                                                'Light Rain _ Windy': 'Light Snow Windy',
                                                                'N_A Precipitation': 'Rain',
                                                                'Heavy Snow _ Windy': 'Snow',
                                                                'Haze _ Windy': 'Haze',
                                                                'Light Rain Shower': 'Rain',
                                                                'Fog _ Windy': 'Fog',
                                                                'T_Storm _ Windy': 'T_Storm',
                                                                'Patches of Fog': 'Fog',
                                                                'Heavy Rain': 'Rain',
                                                                'Thunder _ Windy': 'Thunder',
                                                                'Blowing Dust _ Windy': 'Cloudy Windy',
                                                                'Blowing Snow _ Windy': 'Mostly Cloudy _ Windy',
                                                                'Heavy T_Storm': 'T_Storm',
                                                                'Light Freezing Fog': 'Fog',
                                                                'Heavy T_Storm _ Windy': 'T_Storm',
                                                                'Partial Fog': 'Fog',
                                                                'Widespread Dust _ Windy': 'Fog',
                                                                'Scattered Clouds': 'Cloudy',
                                                                'Drizzle and Fog': 'Fog',
                                                                'Rain _ Windy': 'Windy',
                                                                'Squalls _ Windy': 'Windy',
                                                                'Blowing Snow': 'Snow',
                                                                'Smoke _ Windy': 'Smoke',
                                                                'Light Snow with Thunder': 'Light Snow',
                                                                'Heavy Rain _ Windy': 'Rain',
                                                                'Light Freezing Rain': 'Rain',
                                                                'Clear': 'Fair',
                                                                'Blowing Dust': 'Mostly Cloudy _ Windy',
                                                                'Light Rain Shower _ Windy': 'Light Snow  Windy',
                                                                'Heavy Drizzle': 'Light Drizzle',
                                                                'Thunder _ Wintry Mix': 'Wintry Mix',
                                                                'Light Drizzle _ Windy': 'Light Drizzle',
                                                                'Light Snow Shower': 'Light Snow',
                                                                'Light Thunderstorms and Snow': 'Light Snow',
                                                                'Light Thunderstorms and Rain': 'Light Rain',
                                                                'Freezing Drizzle': 'Drizzle',
                                                                'Squalls': 'Fair _ Windy',
                                                                'Windy': 'Fair _ Windy',
                                                                'Thunderstorm': 'Thunder',
                                                                'Coloudy Wind': 'Mostly Cloudy _ Windy',
                                                                'Drizzle': 'Light Drizzle',
                                                                'Cloudy Windy': 'Mostly Cloudy _ Windy',
                                                                'Clouds': 'Cloudy',
                                                                'Light Snow  Windy': 'Light Snow _ Windy',
                                                                'Thunderstorms and Rain': 'T_Storm',
                                                                'Light Thunderstorm': 'T_Storm',
                                                                'Snow and Thunder': 'Thunder',
                                                                'Mist': 'Cloudy',
                                                                'Heavy Thunderstorms and Rain': 'Light Rain with Thunder',
                                                                'Funnel Cloud': 'Cloudy',
                                                                'Heavy Blowing Snow': 'Snow'
                                                    })

pd.options.display.max_rows=100
CO_crashes_df_updated['Weather_Condition'].value_counts()

In [5]:
le = LabelEncoder()
CO_crashes_df_updated['ID'] = le.fit_transform(df['ID'])
CO_crashes_df_updated.set_index('ID')

CO_crashes_df_updated.info()

(17366, 4)

In [11]:
# Write sorted exchange/gdp data to parquet file
CO_crashes_df_updated.to_csv('./Resources/result_files/step1_build_base_data.csv', sep=',', encoding='utf-8')

In [ ]:

# Calculate Duration
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))
